In [129]:
from PIL import Image, ImageDraw, ImageFont  
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy import interpolate
import cv2

In [156]:
# Parameters
font_type = 'C:/Windows/Fonts/Calibri/calibrib.ttf'
font_size = 40
character = 'A'

In [157]:
# Befehl muss zweimal ausgeführt werden (komisch)
%matplotlib qt
%matplotlib qt

In [158]:
img = Image.new('RGB', (32, 32), color = (0, 0, 0))     
fnt = ImageFont.truetype(font_type, font_size)
d = ImageDraw.Draw(img)
d.text((3,0), character, font=fnt, fill=(255, 255, 255))
img.show() 
# img.save('pil_text_font.bmp')

In [159]:
img = np.array(img)
x,y,z = np.shape(img)

In [160]:
print(x)

32


In [176]:
pixel_numbers = x
x_mesh = np.arange(-1*pixel_numbers+1, pixel_numbers, 1)
y_mesh = np.arange(-1*pixel_numbers+1, pixel_numbers, 1)
xx, yy = np.meshgrid(x_mesh, np.flip(y_mesh,0), sparse=True)
yy = yy*1j
complex_meshgrid = (yy+xx)

print(complex_meshgrid)

[[-31.+31.j -30.+31.j -29.+31.j ...  29.+31.j  30.+31.j  31.+31.j]
 [-31.+30.j -30.+30.j -29.+30.j ...  29.+30.j  30.+30.j  31.+30.j]
 [-31.+29.j -30.+29.j -29.+29.j ...  29.+29.j  30.+29.j  31.+29.j]
 ...
 [-31.-29.j -30.-29.j -29.-29.j ...  29.-29.j  30.-29.j  31.-29.j]
 [-31.-30.j -30.-30.j -29.-30.j ...  29.-30.j  30.-30.j  31.-30.j]
 [-31.-31.j -30.-31.j -29.-31.j ...  29.-31.j  30.-31.j  31.-31.j]]


In [177]:
max_real_complex_meshgrid = int(np.max(complex_meshgrid.real) + 5)
max_imag_complex_meshgrid = int(np.max(complex_meshgrid.imag) + 5)
min_real_complex_meshgrid = int(np.min(complex_meshgrid.real) - 5)
min_imag_complex_meshgrid = int(np.min(complex_meshgrid.imag) - 5)
# print(max_real_complex_meshgrid)
# print(max_imag_complex_meshgrid)
# print(min_real_complex_meshgrid)
# print(min_real_complex_meshgrid)
size_transform_real = max_real_complex_meshgrid - min_real_complex_meshgrid
size_transform_imag = max_imag_complex_meshgrid - min_imag_complex_meshgrid
# print(size_transform_real)
# print(size_transform_imag)

In [178]:
z_transform = np.array(np.empty((size_transform_imag, size_transform_real,3)))
z_transform.fill(np.nan) # nan werden später interpoliert

In [179]:
for i in range(pixel_numbers):
    for j in range(pixel_numbers):
        tmp_real = int(complex_meshgrid[i,j].real - min_real_complex_meshgrid)
#         print(tmp_real)
        tmp_imag = int(complex_meshgrid[i,j].imag - min_imag_complex_meshgrid)
#         print(tmp_imag)
        z_transform[tmp_imag, tmp_real] = img[i,j] 

In [180]:
int(complex_meshgrid[0,0].imag)

31

In [181]:
plt.imshow(z_transform)

In [182]:
z_transform_mask = deepcopy(z_transform)
z_transform_mask[z_transform_mask>0] = 0
where_are_NaNs = np.isnan(z_transform)
z_transform_mask[where_are_NaNs] = 255
# print(z_transform)
z_transform[where_are_NaNs] = 0

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [183]:
# cv2.inpaint benötigt arrays vom typ uint8
# print(z_transform_mask.dtype)
# print(z_transform.dtype)

mask = np.uint8(z_transform_mask)
img = np.uint8(z_transform)

# print(mask.dtype)
# print(img.dtype)

dst_TELEA = cv2.inpaint(img, mask[:,:,0], 3, cv2.INPAINT_TELEA)
dst_NS = cv2.inpaint(img, mask[:,:,0], 3, cv2.INPAINT_NS)

In [184]:
plt.figure('dst_NS')
plt.imshow(dst_NS/255)

0.0015926529164868282